# 信息检索系统第三次作业 向量空间模型
2012522 郭坤昌

## 项目说明
实现自定义组合域上的查询功能。每篇文档有三个域，title，author和body。对于每篇文档，分别对三个域上的词项计算tf-idf。查询时首先判定在哪些组合域上进行查询，使用向量空间模型计算对应域上查询的相似度，最终将不同域上的相似度进行线性组合得到总的文档相似度。

### 数据预处理


In [1]:
import os
import math
import numpy as np

- 移除标点和数字并转化为小写

In [2]:
def remove_punctuation(text):
    punctuation = "0123456789!()-[]{};:'\"\,<>./?@#$%^&*_~"
    for char in text:
        if char in punctuation:
            text = text.replace(char, "")
    return text.lower()

In [3]:
# 测试样例
print(remove_punctuation("Hello! I'm a student. I am 20 years old. That's my school NKU."))

hello im a student i am  years old thats my school nku


- 读取数据，并划分为title，author，body三个域

In [4]:
dataset_path = os.path.join('dataset')
filenames = os.listdir(dataset_path)
title = []
author = []
body = []
for filename in filenames:
    title.append(filename.split('.')[0])    # 获取‘.txt’之前的文件名
    with open(os.path.join(dataset_path, filename), 'r') as f:
        author.append(f.readline().strip('Author: |\n'))    # 获取作者
        body.append(f.read())    # 读取正文

In [5]:
# 测试
print(title)
print(author)
print(body)

['A Drinking Song', 'Aedh wishes for the Cloths of Heaven', 'Down by the Salley Gardens', 'Freedom', 'Leave This', 'Tonight I Can Write', 'Walk with Me in Moonlight', 'When you are old', 'Where the Mind is Without Fear']
['William Butler Yeats', 'William Butler Yeats', 'William Butler Yeats', 'Rabindranath Tagore', 'Rabindranath Tagore', 'Pablo Neruda', 'Leon Knig', 'William Butler Yeats', 'Rabindranath Tagore']
["Wine comes in at the mouth\nAnd love comes in at the eye;\nThat's all we shall know for truth\nBefore we grow old and die.\nI lift the glass to my mouth,\nI look at you, and I sigh.", "Had I the heavens' embroidered cloths,\nEnwrought with golden and silver light,\nThe blue and the dim and the dark cloths\nOf night and light and the half light,\nI would spread the cloths under your feet:\nBut I, being poor, have only my dreams;\nI have spread my dreams under your feet;\nTread softly because you tread on my dreams.", 'Down by the salley gardens my love and I did meet;\nShe pas

- 统计得到tf和df

In [6]:
def get_tf_df_table(text_list, doc_num):
    tf_table = {}
    df_table = {}
    for i in range(doc_num):
        tf_table[i] = {}
        tokens = remove_punctuation(text_list[i]).split()
        for token in tokens:    # 对于文章上的某个域，如果存在某个单词，则tf项加一，否则置一
            if token in tf_table[i]:
                tf_table[i][token] += 1
            else:
                tf_table[i][token] = 1
        unique_tokens = set(tokens)    # 对于文章上的某个域，如果存在某个单词，则df项加一，否则置一
        for token in unique_tokens:
            if token in df_table:
                df_table[token] += 1
            else:
                df_table[token] = 1
    return tf_table, df_table

In [7]:
title_tf_table, title_df_table = get_tf_df_table(title, len(filenames))
author_tf_table, author_df_table = get_tf_df_table(author, len(filenames))
body_tf_table, body_df_table = get_tf_df_table(body, len(filenames))

In [8]:
# 测试
print(title_tf_table)
print(title_df_table)

{0: {'a': 1, 'drinking': 1, 'song': 1}, 1: {'aedh': 1, 'wishes': 1, 'for': 1, 'the': 1, 'cloths': 1, 'of': 1, 'heaven': 1}, 2: {'down': 1, 'by': 1, 'the': 1, 'salley': 1, 'gardens': 1}, 3: {'freedom': 1}, 4: {'leave': 1, 'this': 1}, 5: {'tonight': 1, 'i': 1, 'can': 1, 'write': 1}, 6: {'walk': 1, 'with': 1, 'me': 1, 'in': 1, 'moonlight': 1}, 7: {'when': 1, 'you': 1, 'are': 1, 'old': 1}, 8: {'where': 1, 'the': 1, 'mind': 1, 'is': 1, 'without': 1, 'fear': 1}}
{'song': 1, 'drinking': 1, 'a': 1, 'the': 3, 'heaven': 1, 'wishes': 1, 'for': 1, 'aedh': 1, 'of': 1, 'cloths': 1, 'by': 1, 'down': 1, 'gardens': 1, 'salley': 1, 'freedom': 1, 'leave': 1, 'this': 1, 'can': 1, 'write': 1, 'tonight': 1, 'i': 1, 'with': 1, 'walk': 1, 'me': 1, 'moonlight': 1, 'in': 1, 'you': 1, 'when': 1, 'are': 1, 'old': 1, 'without': 1, 'is': 1, 'fear': 1, 'where': 1, 'mind': 1}


### 向量空间模型

- 计算tf-idf

In [9]:
def tf_idf(tf_table, df_table, doc_num):
    tf_idf_table = {}
    for i in range(doc_num):
        tf_idf_table[i] = {}
        for token in tf_table[i]:
            tf_idf_table[i][token] = tf_table[i][token] * math.log(doc_num / df_table[token])
    return tf_idf_table

In [10]:
title_tf_idf_table = tf_idf(title_tf_table, title_df_table, len(filenames))
author_tf_idf_table = tf_idf(author_tf_table, author_df_table, len(filenames))
body_tf_idf_table = tf_idf(body_tf_table, body_df_table, len(filenames))

In [11]:
# 测试
body_tf_idf_table

{0: {'wine': 2.1972245773362196,
  'comes': 4.394449154672439,
  'in': 0.5026288565618123,
  'at': 6.591673732008658,
  'the': 0.0,
  'mouth': 4.394449154672439,
  'and': 0.35334910696915034,
  'love': 1.0986122886681098,
  'eye': 2.1972245773362196,
  'thats': 2.1972245773362196,
  'all': 1.5040773967762742,
  'we': 4.394449154672439,
  'shall': 2.1972245773362196,
  'know': 2.1972245773362196,
  'for': 1.0986122886681098,
  'truth': 1.5040773967762742,
  'before': 1.0986122886681098,
  'grow': 1.5040773967762742,
  'old': 1.5040773967762742,
  'die': 2.1972245773362196,
  'i': 1.7633599947063572,
  'lift': 2.1972245773362196,
  'glass': 2.1972245773362196,
  'to': 1.5040773967762742,
  'my': 0.4054651081081644,
  'look': 1.5040773967762742,
  'you': 0.8109302162163288,
  'sigh': 2.1972245773362196},
 1: {'had': 1.5040773967762742,
  'i': 2.3511466596084762,
  'the': 0.0,
  'heavens': 2.1972245773362196,
  'embroidered': 2.1972245773362196,
  'cloths': 6.591673732008658,
  'enwrought'

- 向量归一化，避免查询时重复计算
> 直接应用在文档向量的tf-idf记录表上

In [12]:
def norm(tf_idf_table, doc_num):
    for i in range(doc_num):
        length = np.linalg.norm(list(tf_idf_table[i].values()))
        normed = list(tf_idf_table[i].values()) / length
        tf_idf_table[i] = dict(zip(tf_idf_table[i].keys(), normed))
            

In [13]:
norm(title_tf_idf_table, len(filenames))
norm(author_tf_idf_table, len(filenames))
norm(body_tf_idf_table, len(filenames))

In [14]:
# 测试
print(author_tf_idf_table)

{0: {'william': 0.5773502691896258, 'butler': 0.5773502691896258, 'yeats': 0.5773502691896258}, 1: {'william': 0.5773502691896258, 'butler': 0.5773502691896258, 'yeats': 0.5773502691896258}, 2: {'william': 0.5773502691896258, 'butler': 0.5773502691896258, 'yeats': 0.5773502691896258}, 3: {'rabindranath': 0.7071067811865476, 'tagore': 0.7071067811865476}, 4: {'rabindranath': 0.7071067811865476, 'tagore': 0.7071067811865476}, 5: {'pablo': 0.7071067811865476, 'neruda': 0.7071067811865476}, 6: {'leon': 0.7071067811865476, 'knig': 0.7071067811865476}, 7: {'william': 0.5773502691896258, 'butler': 0.5773502691896258, 'yeats': 0.5773502691896258}, 8: {'rabindranath': 0.7071067811865476, 'tagore': 0.7071067811865476}}


- 向量空间模型计算相似度\
文档向量已经经过了归一化，此时只需计算查询向量和归一化文档向量的点积即可

In [15]:
def sim(tf_idf_table, df_table, query, doc_num):
    query_tokens = remove_punctuation(query).split()    # 获取查询单词列表
    query_tf = {}   # 计算查询的tf
    for token in query_tokens:
        if token in query_tf:
            query_tf[token] += 1
        else:
            query_tf[token] = 1
    sim_list = []   # 对所有文档进行相似度计算得到相似度列表
    for i in range(doc_num):
        sim_list.append(0)
        for token in query_tokens:
            if token in tf_idf_table[i]:    # 如果查询单词在文档中出现过，则计算相似度
                query_token_tf_idf = query_tf[token] * math.log(doc_num) / df_table[token]   # 这里仍使用文档集合的df，认为文档足够多，使得df不会因为查询而变化较大
                sim_list[i] += query_token_tf_idf * tf_idf_table[i][token]
    return sim_list

> 下面的句子摘自‘A Drinking Song’，对应下标为4，得分最高，计算正确

In [16]:
# 测试
test_query = "That's all we shall know for truth"
sim_list = sim(body_tf_idf_table, body_df_table, test_query, len(filenames))
print(sim_list)
print(filenames[np.argmax(sim_list)])

[2.1902784903035246, 0, 0, 0.040765923471441486, 0.12596684694753174, 0.04119528753537085, 0, 0, 0.09328191349114537]
A Drinking Song.txt


- 对查询结果进行排序并展示，得分为0则不予展示

In [17]:
def show_result(sim_list, titles):
    result_dict = dict(zip(sim_list, titles))
    sorted_dict = sorted(result_dict.items(), key=lambda x: x[0], reverse=True)
    rank = 1
    for item in sorted_dict:
        if item[0]!= 0:
            print("{rank}\tSim: {sim:.2f}\tDoc: < {doc} >".format(rank = rank, sim = item[0], doc = item[1]))
            rank += 1
    

In [18]:
# 测试
show_result(sim_list, title)

1	Sim: 2.19	Doc: < A Drinking Song >
2	Sim: 0.13	Doc: < Leave This >
3	Sim: 0.09	Doc: < Where the Mind is Without Fear >
4	Sim: 0.04	Doc: < Tonight I Can Write >
5	Sim: 0.04	Doc: < Freedom >


## 组合域查询的实现

- 设置各域的权重，即应用组合查询
- 若对应域没有输入，则对应域上的相似度加权一定为0，即不在对应域上查找

In [19]:
def search(query, titles, doc_num):
    # 权重
    title_weight = 0.2
    author_weight = 0.2
    body_weight = 0.6
    # 计算相似度
    title_sim_list = sim(title_tf_idf_table, title_df_table, query['title'], doc_num)
    author_sim_list = sim(author_tf_idf_table, author_df_table, query['author'], doc_num)
    body_sim_list = sim(body_tf_idf_table, body_df_table, query['body'], doc_num)
    # 线性组合相似度
    sim_list = [title_sim_list[i]*title_weight + author_sim_list[i]*author_weight + body_sim_list[i]*body_weight for i in range(doc_num)]
    show_result(sim_list, titles)

- 以下为了方便，提供查询的格式化输入，可根据查询需要修改

In [24]:
query = {   
            "title":"Drinking", 
            "author":"Rabindranath", 
            "body":"Wine burden"
        }

In [25]:
search(query, title, len(filenames))

1	Sim: 0.48	Doc: < A Drinking Song >
